# Introduction into web scraping

## HTTP Request

HTTP zahtjev je protokol na principu kojeg danas funkcionira web

HTTP pozivima (requestima) mozemo preuzimati sadrzaj s interneta

HTTP protokol se od
 - metode (GET, PUT, POST, DELETE, etc.)
 - destinacije (url ili ip adresa)
 - headeri (forme, kolacici, tokeni za autorizacije, itd.)
 
<img src="assets/http-com.png" alt="Image" width="500">


## HTML (Hypertext Markup Language) Dokumenti

U nasem slucaju server ce nam vracati HTML dokumente - u principu to je tree datastructure (document tree)
<img src="assets/html-tree.png" alt="Image" width="500">

Kad istrazujes, nemoj se zamarati CSS-om, JS-om, nas zanimaju cisti podatci a ne ponasanje i stiliziranje stranice

Sastoji se od raznih "tagova" koji sluze internet preglednicima da prikazuju te informacije na razlicite nacine

Za potrebu obrade podataka, nas ne zanima koji tagovi postoje, vec cemo ih tretirati kao 'noise' u tekstu

HTML dokument je kao bilo koji drugi dokument poput PDF, CSV, .PY, itd. te ima svoj format


### Primjer 1
```html
<!DOCTYPE html>
<html>
  <head>
    <title>TEXT</title>
  </head>
    
  <body>
    <h1>TEXT2</h1>
    <p>TEXT3</p>
  </body>
</html>

```

Ako iz ovog primjera zelimo izvuci TEXT2 vrijednost morali bi napraviti sljedece:

- trazimo HMTL tag
- trazimo child koj se zove "body"
- u njemu trazimo child koj se zove "h1"


### Primjer 2
```html
<!DOCTYPE html>
<html>
  <head>
    <title>TEXT</title>
  </head>
    
  <body>
      <list>
          <item>Auto 1</item>
          <item>Auto 2</item>
          <item>Auto 3</item>
          <item>Auto 4</item>          
      </list>
  </body>
</html>
```

Ako zelimo iz ovog dokumenta izvuci sve automobile, moramo napraviti sljedece
 - trazimo element koj se zove list
 - napravimo petlju za svaki child u tom elementu
 - for each child write value to database / csv / json
 
 
### Primjer 3
```html
<!DOCTYPE html>
<html>
  <head>
    <title>TEXT</title>
  </head>
    
  <body>
      <div id='list'>
          <div>Auto 1</div>
          <div>Auto 2</div>
          <div>Auto 3</div>
          <div>Auto 4</div>
      </div>
  </body>
</html>
```

Ako zelimo iz ovog dokumenta izvuci sve automobile nemozemo vise traziti element koj se zove div, zato sto ima hrpa div elementa, ali neki elementi sadrze atribute. U ovom slucaju, nasa lista sadrzi atribut koj se zove ID te mozemo elemente pretrazivati i pod atributima. 

Ne-pisano pravilo je da je ID element uvijek unikatan u svakom elementu, znaci nemoze postojat vise atributa s jednakom vrjednosti za ID

 - trazimo element koji sadrzi atribut ID s vrjednosti list
 - napravimo petlju za svaki child u tom elementu
 - zapisemo u database / csv / json

## Primjer HTTP Poziva

In [1]:
# Import requests library - used to send HTTP requests
import requests


# koristimo requests da posaljemo GET request na zeljeni URL 
url = "https://www.index.hr/oglasi/osobni-automobili/gid/27?elementsNum=100"
response = requests.get(url)

# Isprintat status zahtjeva (200 = OK, 404 = NOT FOUND, 500 = SERVER ERROR, etc.)
display(response)

print("\n Prvih 200 znakova primljenog dokumenta")
display(response.text[:200])

<Response [200]>


 Prvih 200 znakova primljenog dokumenta


'<!DOCTYPE html>\r\n<html translate="no">\r\n\t<head>\r\n        <meta charset="utf-8" />\r\n\t\t<meta property="fb:app_id" content="1697489853797759"/>\r\n\r\n\t\t\r\n\r\n\t\t\t<title>Rabljena vozila na prodaju | Auto-moto o'

## Prikazati HTML kao sto bi ga preglednik prikazao

In [ ]:
from IPython.display import HTML

display(HTML(response.text))


In [3]:
# Import bs4 library - used to parse HTML and XML documents
from bs4 import BeautifulSoup

# Biblija - https://beautiful-soup-4.readthedocs.io/en/latest/

# Inicijalizirati http parser i predati mu response.text (moze se napravit i xml parser)
# nas dokument je spremljen u memoriji u varijabli response.text !
soup = BeautifulSoup(response.text, "html.parser")

# Prisjetimo se gore Primjera 3, trebamo pronaci element u kojem se nalaze svi automobili
## Za svaki child elementa (svaki auto) stavit u petlju i pisati u file

# Saznati cemo koji element nam treba tako da otvorimo stranicu u web pregledniku
# Desni klik -> inspect element

# nas element ima atribut koj se zove class i ima vrjednost list <div class='list'>
children = soup.find_all("div", class_="OglasiRezHolder")

childrenWithoutBanners = soup.select("div.OglasiRezHolder:not(.oglasiHolderBanners)")

# Isprintat 3. child
display(children[3])
    


<div class="OglasiRezHolder">
<a class="result" href="https://www.index.hr/oglasi/renault-twingo-1-5-dci/oid/4022552">
<figure class="result_photo" data-x="2">
<img alt="Renault Twingo 1.5 DCI" src="https://www.index.hr/oglasi/userdocsimages/oglas/_2023//5//12/4022552/twingo2-120520231212486547.jpg?preset=oglas-slika-view"/>
</figure>
<span class="result_body">
<span class="head">
<span class="title px18">
Renault Twingo 1.5 DCI
<span class="addFavorit icon-heart tooltip_bottom" title="Spremi oglas">
</span>
</span>
<span class="lead" data-fff="bb"></span>
<ul class="tags hide-on-small-only"><li>
Godište
 : 
2012
</li>
<li>
km
 : 
137.770
</li>
<li>Starost : Rabljeno</li>
<li>
kW
 : 
55
</li>
</ul>
</span>
<span class="foot">
<ul class="info">
<li class="icon-marker">Ličko-senjska</li>
<li class="icon-time">Objavljeno prije 11 min</li>
</ul>
<span class="price"><span>5.500 €</span> ~ 41.440 kn</span></span>
</span>
</a>
</div>

# Pronalazak vrijednosti u child elementima

Svaki child element u nasoj listi je poprilicno kompliciran, sadrzi slike, linkove, formatiranje i razno razne tagove (elemente) koji nam nisu korisni

Ovo nebi trebalo bit tesko za procesirat, svaka vrijednost ima unikatni identifikator (title, lead) osim tagova.

Tagovi su zeznuti zato sto nemozemo atribuirat vrijednost po samom tagu, vec moramo pogledati vrijednost taga i procjeniti da li vrijednost sadrzi tekst "godiste" ili "km" te na osnovu toga odluciti gdje pripada taj podatak

```html
<div>
    <a class="result" href="https://www.index.hr/oglasi/bmw-serija-5-530d-automatic/oid/4013168">
        <div>
            <span class="title">NAZIV_AUTA</span>
        </div>

        <span class="lead">OPIS_AUTA</span>

        <ul class="tags">
            <li>Godiste: 2021</li>
            <li>km: 20.000</li>
            <li>Starost: Rabljeno</li>
            <li>kW: 100</li>
        </ul>

        <span class='price'>
            <span>CIJENA_AUTA</span>
        </span>

        <li class='icon-marker'>::before ZUPANIJA</li>
    </a>
</div>
```

In [8]:
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Create an empty dataframe with columns
df = pd.DataFrame(columns=["title", "price", "url"])

def findData(child):
    title = child.find("span", class_="title")
    price = child.find("span", class_="price")
    
    euros = price.text.split("€")[0]
    return {"title": title.text, "price": euros, "url": "www.google.com"}

# Za svaki child kreirat podatke i ubacit u dataframe
for child in children:
    
    arrayKlasa = child.get('class')
    
    # Ako je kolicina klasa vise od 1
    if len(arrayKlasa) > 1:
        print(len(arrayKlasa))
        # Continue krece u iducu iteraciju petlje bez da executa kod ispod sebe
        # Znaci preskocit cemo row = findData() i df.append()
        continue

    row = findData(child)
    df = df.append(row, ignore_index=True)
    
display(df)

# Pisanje podataka u file
df.to_csv('auti.csv')

2
3
3
3
3
3
3
3
3
3
3


,title,price,url
0,\r\nOpel Astra 1.4\r\n\n\n,750,www.google.com
1,\r\nAlpina 20 zamjena za auto ili skuter\r\n\n\n,2.000,www.google.com
2,\r\nRenault Twingo 1.5 DCI\r\n\n\n,5.500,www.google.com
3,\r\nAudi A6 2.0tdi ultra\r\n\n\n,18.900,www.google.com
4,\r\nFiat Panda 1.2 ☆ 4 x 4 ☆ CLIMBING ☆KLIMA☆...,5.400,www.google.com
...,...,...,...
95,\r\nAudi A4 B7 3.0 TDI QUATTRO\r\n\n\n,5.650,www.google.com
96,"\r\nRenault Espace 1.6dci, 2016g.reg 8/2023, f...",15.999,www.google.com
97,\r\n⭕ Ford Fiesta 1.1 Kupljen nov u HR Samo ...,11.900,www.google.com
98,\r\nVW PASSAT 2.0 TDI*110KW/150KS*REG 3/24*AUT...,16.000,www.google.com


# Zadaća

### 1. Zadatak
Zasad smo kreirali dataframe s nazivom i cijenom
Treba izvuci sve moguce vrijednosti koje mozemo pronaci za neki auto

Hint: Trebat ce cesto provjeravati da li je vrjednost None, nisu konzistentni podatci
Hint2: Trebat ce nam i URL da pronadjemo vise podataka o svakom pojedinacnom autu (`<a class="result" href="URL_NA_POJEDINACNI_AUTO">`), kako ces izvaditi atribut?
        
### 2. Zadatak
Ovaj URL ce nam vratit dokument s 1. stranicom automobila s 10 rezultata. 
`url = "https://www.index.hr/oglasi/osobni-automobili/gid/27"`

To je defaultna vrijednost. Neke defaultne vrijednosti web stranica mozemo mijenjati pomocu query parametra u URLu. To saznamo tako da koristimo web stranicu i gledamo kako se URL mijenja. Promjene u URLu su u HTTP protokolu nazvani GET parametri.

Ukoliko dodamo GET parametar `elementsNum=100` onda cemo dobiti natrag 100 rezultata umjesto defaultnih 10.

`url_100_elements = "https://www.index.hr/oglasi/osobni-automobili/gid/27?elementsNum=100"`

Ukoliko dodamo jos jedan parametar `num=2` onda cemo dobiti 2. stranicu s autima

`url_100_elements = "https://www.index.hr/oglasi/osobni-automobili/gid/27?elementsNum=100&num=2"`

Razmisliti kako skinuti podatke s iducih stranica

hint: for petlja
hint2: counter

In [5]:
from time import sleep

print(1)

sleep(5)

print(2)

1
2
